In [93]:
import requests
import json
import chardet
import re
import pandas as pd
import numpy as np
from pprint import pprint


In [94]:
# 1.根据bvid请求得到cid

def get_cid():
    url = 'https://api.bilibili.com/x/player/pagelist?bvid=BV1MN4y177PB'
    res = requests.get(url).text
    json_dict = json.loads(res)
    pprint(json_dict)
    return json_dict["data"][0]["cid"]
cid = get_cid()
print(cid)



{'code': 0,
 'data': [{'cid': 783037295,
           'dimension': {'height': 1080, 'rotate': 0, 'width': 1920},
           'duration': 688,
           'first_frame': 'http://i0.hdslb.com/bfs/storyff/n220724a23lei8ykkbd5bxhmzwye2gcu_firsti.jpg',
           'from': 'vupload',
           'page': 1,
           'part': '回村三天，二舅治好了我的精神内耗',
           'vid': '',
           'weblink': ''}],
 'message': '0',
 'ttl': 1}
783037295


In [95]:
# 2.根据cid请求弹幕，解析弹幕得到最终的数据

def get_data(cid):
    final_url = "https://api.bilibili.com/x/v1/dm/list.so?oid=" + str(cid)
    final_res = requests.get(final_url)
    final_res.encoding = chardet.detect(final_res.content)['encoding']
    final_res = final_res.text
    print(final_res)
    pattern = re.compile('<d.*?>(.*?)</d>')
    match = re.compile(r'<d\s+[^>]*\bp="([^"]*)"')
    pa = match.findall(final_res)

    data = pattern.findall(final_res)
    danmu_time = [float(item.split(',')[0]) for item in pa]
    danmu_mode = [float(item.split(',')[1]) for item in pa]
    danmu_size = [float(item.split(',')[2]) for item in pa]
    danmu_color = [float(item.split(',')[3]) for item in pa]
    danmu_abstime = [float(item.split(',')[4]) for item in pa]
    danmu_pool = [float(item.split(',')[5]) for item in pa]
    danmu_id = [(item.split(',')[6]) for item in pa]
    danmu_rowid = [float(item.split(',')[7]) for item in pa]
    return (data, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid)


(danmu_text, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid) = get_data(cid)

<?xml version="1.0" encoding="UTF-8"?><i><chatserver>chat.bilibili.com</chatserver><chatid>783037295</chatid><mission>0</mission><maxlimit>1500</maxlimit><state>2</state><real_name>0</real_name><source>k-v</source><d p="494.60500,5,25,15138834,1658716910,0,b4109bf0,1104156486730520320,10">《我的超人二舅》</d><d p="516.84800,5,25,16777215,1658753207,0,327e2678,1104460967271513600,10">《拆上几次也会修了》</d><d p="389.67500,5,25,16777215,1658879835,0,ed15abb8,1105523199019406848,10">马上6789了，更酷</d><d p="455.82100,5,25,16707842,1658828961,0,371352d5,1105096440423717120,10">这在走近科学能拍十集</d><d p="502.56100,5,25,15138834,1658833687,0,c5cbe003,1105136082963577600,10">《点 名 表 扬》</d><d p="475.39500,5,25,16707842,1658832868,0,c268f3f9,1105129210478899968,10">如若没有那场意外，他现在该过着怎样的生活。</d><d p="577.55400,5,25,16646914,1658829922,0,e840ecf6,1105104503495153664,10">现在他是余华笔下坎坷而伟大的普通人</d><d p="171.50900,1,25,16777215,1658845198,0,aaaffccb,1105232645329087744,10">“不要光赞美高耸的山峰，平原和丘陵也一样不朽”</d><d p="529.24000,5,25,16777215,16587613

In [96]:
# 分割弹幕为八个部分
# 视频持续时间
max_time = max(danmu_time)
divided_danmu = []
for i in range(8):
    divided_danmu.append([])
    time_start = max_time * (i/8)
    time_stop = max_time * ((i+1)/8)
    for j in range(len(danmu_time)):
        if time_start <= danmu_time[j] <= time_stop:
            divided_danmu[i].append(danmu_text[j])
print(divided_danmu[0])

['对不起，第一次看到这个视频我划走了，现在我回来了', '火了赞我', '四针扎没了一个天才少年', '从苦难者身上汲取力量，又有谁去在意苦难者的苦难？', '昨天晚上班主任刚在班里面放过，今天回来就刷到了？！', '《人生海海》的感觉', '没绷住', '二舅是一位伟大的“中医”，治好了千万人恐惧、担忧、自卑、孤独的心理疾病。', '火去视频号儿了都！衣戈强！！', '我第一次见到如此恐怖的硬币数', '为小侄子点赞', '太棒了 我因此回答了60+问题', '一键三连数疯狂上涨', '我就不三连了啊，先去投俩币再点赞收藏', '这段有种余华的感觉', '《看会了》', '欢迎来到币比赞多系列', '为二舅献上人生第一个一键三连', '第一次见那么多人观看', '看第一遍懵懂，第二遍触动，第三遍敬仰、感动', '敬二舅', 'popping或者poppin，不是poping', '欢迎观看2022年b站最佳视频', '2022年最伟大的作品', '他大舅他二舅都是他舅，高桌子低板凳都是木头', '1200一条烟', '我觉得不如用土方法……', '二舅的针 是什么针？', '希望这视频的播放不会引来一些别有用心的人，打扰二舅宁静的生活。', '大家奋斗不是为了像二舅一样吃苦，取得胜利是精神和物质的双重丰收，而不是无奈的自我升华。', '二舅与姥姥的故事由此开始...', '感觉河北邯郸', '回湖南32天，决定了我的基本路线', '夜里两点九千加在看！', '残废的二舅是最快乐的人，讽刺不讽刺？', '要愤怒，不要沉默', '敬二舅！！！[666]谢谢作者拍摄与分享，你写的关于“遗憾”的结语，很受用！！', '真是一个让人感到绝望的视频！', '年度宝藏视频！！直接封神之作', '回来点个三连', '入站必刷', '敬二舅。', '敬二舅！', '可恶，为什么一个视频只能投两个币', '苦难无法治愈苦难，二舅苦难的根源是什么', '不要打扰二舅！', '欢迎来到币比赞多系列', '这个视频很可能，变成点赞量和硬币量最多的视频', '敬二舅', '见证二舅打破全站投币量记录', '看完这个视频我想起一句话那就是这个世界只有一种英雄主义，就是认清真相之后依然热爱生活', '啊                                              

In [97]:
# export csv
arr = np.array(divided_danmu[0])
df = pd.DataFrame(arr)
df.columns =
df.to_csv("0.csv")
arr = np.array(divided_danmu[1])
df = pd.DataFrame(arr)
df.to_csv("1.csv")
arr = np.array(divided_danmu[2])
df = pd.DataFrame(arr)
df.to_csv("2.csv")
arr = np.array(divided_danmu[3])
df = pd.DataFrame(arr)
df.to_csv("3.csv")
arr = np.array(divided_danmu[4])
df = pd.DataFrame(arr)
df.to_csv("4.csv")
arr = np.array(divided_danmu[5])
df = pd.DataFrame(arr)
df.to_csv("5.csv")
arr = np.array(divided_danmu[6])
df = pd.DataFrame(arr)
df.to_csv("6.csv")
arr = np.array(divided_danmu[7])
df = pd.DataFrame(arr)
df.to_csv("7.csv")

In [98]:
# 3.保存弹幕列表

def save_to_file(data):
    with open("dan_mu.txt", mode="w", encoding="utf-8") as f:
        for i in data:
            f.write(i)
            f.write("\n")
save_to_file(data)


TypeError: write() argument must be str, not float

In [ ]:
# 1 导入相关库
import pandas as pd
import jieba
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from imageio import imread

import warnings
warnings.filterwarnings("ignore")

# 2 读取文本文件，并使用lcut()方法进行分词
with open("dan_mu.txt",encoding="utf-8") as f:
    txt = f.read()
txt = txt.split()
data_cut = [jieba.lcut(x) for x in txt]
data_cut
# 3 读取停用词
stop = [" ","道","说道","说",'了']
# 4 去掉停用词之后的最终词
s_data_cut = pd.Series(data_cut)
all_words_after = s_data_cut.apply(lambda x:[i for i in x if i not in stop])
# 5 词频统计
all_words = []
for i in all_words_after:
    all_words.extend(i)
word_count = pd.Series(all_words).value_counts()
# 6 词云图的绘制
# 1）读取背景图片


# 2）设置词云参数
wc = WordCloud(font_path="C:/Users/Windows/fonts/simhei.ttf",
               background_color="white",
               max_words=2000,
               max_font_size=200,
               random_state=42
              )
wc2 = wc.fit_words(word_count)

# 3）绘制词云图
plt.figure(figsize=(16,8), dpi=300)
plt.imshow(wc2)
plt.axis("off")
plt.show()
wc.to_file("ciyun.jpg")

